# Documentation
This file contains the docs for objects and functions that interface with the user.

### class pylove_state
______________________________________________________
Object instance for quantum states with certain stabilizer properties. The stabilizers can be known but the state can be a mixed state of density matrices with any combination of syndromes (sign flips of the stabilizers). The range of syndromes can however also be restricted, as would be the case after postselection experiments. The density matrix could be restricted to only hold states in the code space, or even just pure states of it, stored as vectors. As the stabilizers are known, only the logical representation of the states are stored densely, while stabilizers are stored symbolically along with instructions on how the logical representations are obtained.


|Attribute|Description|
|---|---|
| state | Returns the reconstructed density matrix. If `self.mode == 'all'`, the output is an array of the shape `( *A, *B)`, where `A = (2, 2, 2, ... , 2)` and `B = (2 ** n, 2 ** n)` with `n = self.n_log_qubits` being the number of logical qubits. The idea is that this matrix prompts a density matrix block associated with a syndrome pattern, a tuple `syndr`, by using it as an input; `self.state[syndr]`. If `self.mode == 'code'`, the output is a two-dimensional array: the density matrix block associated with the code space. If `self.mode == 'custom'`, the output is of type dictionary with syndrome patterns as keys. The syndrome patterns are a version of the syndromes specified in the input `block_numbers`, but updated to fit the set of stabilizer generators in `self.stabs`. If `self.mode == 'vector'`, the output is a `numpy.ndarray` vector.|
|mode| Outputs a string indicating the format of `self.state`. Possible configurations are `'all'`, `'code'`, `'custom'` or `'vector'`.|
| ideal_state | The logical state if the circuit was noiseless, stored as a vector.|
| n_physical_qubits | Number of physical qubits in the state.|
| n_log_qubits | Number of logical qubits; the size of the computational subspace.|
| circuit_depth | Depth of the quantum circuit (including the state preparation) that created the state.|
| stabs | List of signed Pauli strings of type QubitOperator; the updated list of stabilizer generators.|
| n_entries | Number of shots that were binned to reconstruct the density matrix. The difference between the total number of shots and `self.n_entries` has been discarded during the simulation.|
| qubit_order | List of integers and flags 'rm', defining the relationship between physical and logical qubits. If `self.qubit_order[m] == n`, where `n` is an integer, then the `m`th physical qubit becomes the `n`th logical qubit. If `self.qubit_order[m] == 'rm'`, then the qubit is removed.|
| fixed_positions | List of integers, indicating which physical qubit is fixed by which stabilizer. The `n`-th stabilizer `self.stabs[n]` fixes the `fixed_positions[n]`-th physical qubit.|
| fixed_ops | String of characters `'X'`, `'Y'` and `'Z'` indicating the type of Pauli operators fixed by each stabilizer. The `n`-th stabilizer `self.stabs[n]` fixes the `fixed_positions[n]`-th physical qubit to the operator `fixed_ops[n]`.|
| other_ops | String of characters `'X'`, `'Y'` and `'Z'` indicating the type of Pauli operators fixed by each stabilizer, in addition to `fixed_ops`. To remove the `n`th qubit from a physical operator, it must act trivially on the `n`th qubit. That is, it must act as either the identity or `P`, where `(P, self.fixed_ops[n], self.other_ops[n])`is a permutation of `('X', 'Y', 'Z')`.

$\,$

|Method|Description|
|---|---|
| `self.ideal()` | Creates a `pylove_state` instance of the current instance's ideal version; the state returned by the noiseless version of the quantum circuit. |


### pylove_simulation
______________________________________________________

```python
pylove_simulation(
        stabilizers: Union[Sequence, QubitOperator],
        logical_operators: Sequence,
        quantum_circuit: Sequence,
        shots: int,
        wire_noise: list = noiseless(),
        gate_noise: list = noiseless(),
        state_prep_circuit: Sequence = [],
        mode: str = 'code',
        block_numbers: Sequence = (),
        queue_size: int = 100,
        num_processes: int = 0) -> pylove_state:
```

Simulates a physical quantum circuit acting on quantum state with stabilizers under Pauli noise. In this simulator, noise operators are statistically placed in and around circuit subroutines (according to the noise model), and symbolically reduced to their logical representation: Pauli strings on the computational state space. Logical state vectors and
syndrome patterns constitute shots with which the density matrix is reconstructed. The density matrix itself is block diagonal with respect to stabilizer states of different syndrome patterns, including the code space. The simulation can save any number of these blocks corresponding to different syndrome numbers. Perfect postselection for instance would only require to keep the code space block and the rest can be discarded. The states in every block are logical states of the stabilizer state corresponding to their syndrome pattern. The simulated circuit has two parts, a time evolution / ansatz circuit featuring Pauli string rotation subcircuits and a state preparation routine featuring subcircuits for projective measurements. Besides the stabilizer generators that constrain the system at every point in the circuit, the system is additionally constrained by a number of logical operators (signed Pauli strings) after the state preparation circuit. This extended list of stabilizer generators constrains the state completely, meaning there are as many extended stabilizer generators as there are physical qubits and the computational subspace has zero degrees of freedom. It is possible within this simulator to give the state preparation circuit an entirely new pattern of projective measurements, as long as they span the same eigenspace as the original stabilizers and logical operators.

|Arg| Description|
|---|---|
|stabilizers (`QubitOperator` or `list`)| List of symbolic expressions of type `QubitOperator` (or a `QubitOperator`-typed sum of signed Pauli strings) signifying stabilizer generators.|
|logical_operators (`list`)| List of symbolic expressions of type `QubitOperator`, signifying the logical operators stabilizing the initial state, given in their physical representation.|
|quantum_circuit (`list` or `tuple`)| Sequences of QubitOperator-typed symbolic expressions of the form `QubitOperator(pstring: str, angle: float)` signifying rotation subcircuits of the Pauli strings `pstring` about the angle `angle` in the time evolution/ansatz circuit. The subcircuits are placed into the circuit in the order of this sequence.
|shots (`int`)| Number of shots for the reconstruction of the density matrix. The state preparation and time evolution/ansatz circuit is evaluated at every shot.|
|wire_noise (`list`)| List characterizing the single-qubit noise model. It has the form `[A, B]`, holding two sublists `A` and `B` where `A` is a list of Pauli operators (`QubitOperators`) including the identity, that together with their respective statistical weight in the next sublist form the error channel's Kraus operators. The Pauli operators have the index `0` as a placeholder for the proper label of the qubit the noise acts on. `B` is a list holding float numbers corresponding to the statistical weights of their respective Pauli operators.|
| gate_noise (`list`)| List characterizing the gate noise model. It has the form `[A, B]`, holding two sublists `A` and `B`: `A` is a list of Pauli strings (`QubitOperators`) including the identity, that together with their respective statistical weight in the next sublist form the error channel's Kraus operators. The Pauli strings itself are on two qubits, where the integers `0` and `1` function as placeholders for the labels of control and target qubit, respectively. `B` is a list holding `float` numbers corresponding to the statistical weights of their respective Pauli string.|
|state_prep_circuit (`list`) [optional]| List of symbolic expressions of type `QubitOperator` outlining the schedule for the state preparation circuit. This list contains the signed Pauli strings in the order the measurement sub-circuits are placed into the circuit before the ansatz/time evolution circuit `quantum_circuit` is run. If `state_prep_circuit` is left empty, the schedule is made from appending the inputs for the parameters `stabilizers` and `logical_operators`. The list is empty by default.|
|mode (`str`) [optional]| String of characters, expected to be one of the key words `'code'`, `'all'` or `'custom'` setting the mode of operation. Set to `'code'` by default. This parameter decides which blocks of the density matrix to keep, and determines the output format.  `mode='all'` keeps blocks of all syndromes, and returns the reconstructed density matrix as a high-dimensional array that would return a particular block by taking its syndrome pattern as a parameter, see below. `mode='code'` only keeps the code block which is returned as a two-dimensional array. `mode='custom'` keeps only the blocks with syndrome numbers specified in `block_numbers`, and the reconstructed density matrix is returned as a dictionary, with the transformed syndromes as keys. These syndromes are generally different from the ones in `block_numbers` due to a necessary rearrangement of the stabilizer generators, that can be retraced with the other outputs of this routine.|
|block_numbers (`list` or `tuple`) [optional]| Sequence of tuples containing integers `0` and `1`. A collection of syndrome patterns saved in the reconstructed density matrix when `mode` is set to `'custom'`. Empty by default. This `block_numbers` is ignored if `mode` is set to anything else.|
|queue_size (`int`) [optional]| Maximum number of state vectors and syndrome patterns stored in a queue filled by parallel processing waiting to be popped by the `numpy` part of the simulation updating the density matrix. The `numpy` part of the simulator can empty the queue several times and `shots` does not need to be an integer multiple of `queue_size`. Set to `100` by default.|
|num_processes (`int`) [optional]| Number of workers in the process pool parallelizing the shots right up to the `numpy` part of the simulation. If `num_processes` is set to zero the number of processes set by a CPU count. Zero by default.|

$\,$

|Returns|Description|
|---|---|
| `pylove_state` | Density matrix object resulting the simulation.| 
    


### postselect
______________________________________________________
```python
postselect(
        state: pylove_state,
        mode: str,
        block_numbers: Sequence = []) -> pylove_state:
```

Restricting a state to a specified set of density matrix sub-blocks, equivalent to a post-selection experiment.

|Arg| Description|
|---|---|
|state (`pylove_state`)| Input state, where `mode` is not already `code`.|
|mode (`str`)| New `mode` for the output density matrix, characterizing the restriction to a certain set of density matrix sub-block. Expects keywords `'code'` or `'custom'`, where  `mode='code'` only keeps the code block and `mode='custom'` keeps only the blocks with syndrome numbers specified in `block_numbers`.|
|block_numbers (`list` or `tuple`)|Sequence of tuples containing integers `0` and `1`. A collection of syndrome patterns saved in the reconstructed density matrix when `mode` is set to `'custom'`. Empty by default. This `block_numbers` is ignored if `mode` is set to anything else.|

$\,$

|Returns|Description|
|---|---|
| `pylove_state` | Restricted state, normalized to unit trace. Its `mode` matches the function argument, it's `n_entries` is updated to reflect the true content of the remaining blocks.|


### tr
______________________________________________________
```python
tr(
        arg1: Union[QubitOperator, pylove_state],
        arg2: Union[QubitOperator, pylove_state] = QubitOperator(())
        ) -> complex:
```
Trace of a density matrix with an operator (outputting an expectation value) or an ideal state (outputting a fidelity).

|Arg|Description|
|---|---|
|arg1 (`pylove_state` or `QubitOperator`)| Quantum state or physical-level symbolic operator.|
|arg2 (`pylove_state` or `QubitOperator`) [optional]| Quantum state or physical-level symbolic operator.  Note that `arg1` and `arg2` cannot both be mixed states.|

$\,$

|Returns|Description|
|---|---|
|`float`|Fidelity or expectation value, depending on input formats.|